In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [2]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [3]:
train

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,2.0,5.0,4.0,2.0,4.0,USA,12,103,161,2.0,2.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,2.0,2,2.0,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,1.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,1.0,5.0,3.0,4.0,4.0,5.0,4.0,5.0,USA,31,106,179,3.0,2.0,4.0,5.0,4.0,3.0,4.0,1.0,2.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,4.0,1,2.0,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0,USA,17,103,168,1.0,3.0,2.0,5.0,1.0,5.0,3.0,3.0,1.0,1.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,3.0,5.0,USA,14,68,109,1.0,1.0,3.0,5.0,4.0,5.0,5.0,4.0,2.0,1.0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,3.0,2,2.0,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         35452 non-null  int64  
 1   Q1            35333 non-null  float64
 2   Q2            35305 non-null  float64
 3   Q3            35322 non-null  float64
 4   Q4            35327 non-null  float64
 5   Q5            35362 non-null  float64
 6   Q6            35320 non-null  float64
 7   Q7            35310 non-null  float64
 8   Q8            35344 non-null  float64
 9   Q9            35348 non-null  float64
 10  Q10           35232 non-null  float64
 11  Q11           35333 non-null  float64
 12  Q12           35303 non-null  float64
 13  Q13           35356 non-null  float64
 14  Q14           35350 non-null  float64
 15  Q15           35345 non-null  float64
 16  Q16           35367 non-null  float64
 17  Q17           35373 non-null  float64
 18  Q18           35305 non-nu

In [6]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)
train_=train
train_y = train['nerdiness']

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [8]:
train.isna().sum()

Q1               41
Q2               69
Q3               50
Q4               71
Q5               38
Q6               48
Q7               76
Q8               48
Q9               56
Q10              72
Q11              59
Q12              67
Q13              40
Q14              36
Q15              45
Q16              33
Q17              37
Q18              63
Q19              53
Q20              45
Q21              39
Q22              38
Q23              50
Q24              61
Q25              44
Q26              68
introelapse       0
testelapse        0
surveyelapse      0
TIPI1            53
TIPI2            66
TIPI3            79
TIPI4            64
TIPI5            70
TIPI6            62
TIPI7            64
TIPI8            65
TIPI9            64
TIPI10           80
VCL1              0
VCL2              0
VCL3              0
VCL4              0
VCL5              0
VCL6              0
VCL7              0
VCL8              0
VCL9              0
VCL10             0
VCL11             0


In [9]:
############ 이상치 검출 ############


# 피처마다 최대 최소값 확인 -> 
# 딴 컬럼들 말고 수치형변수(introelapse, testelapse, surveyelapse, age, familysize에만 이상치가 있음을 확인)
# print(train_x.max())
# print(train_x.min())



# 위에 적힌 수치형 변수들에 대해서 보편적인 기준으로 이상치 제거
# 기준은 Q3 + 1.5 * IQR보다 높거나 Q1 - 1.5 * IQR보다 낮은 값을 의미함.-> 근데 1.5로하면 이상치 개많음,, 그래서 변경해야댐
#     Q3 : 100개의 데이터로 가정 시, 25번째로 높은 값
#     Q1 : 100개의 데이터로 가정 시, 75번째로 높은 값
#     IQR : Q3 - Q1의 차이



tmp_x = train
df = train[['introelapse','testelapse','surveyelapse','age','familysize']]

def delete_outlier(x):
    sum = 0
    global df,tmp_x
    q3 = train.quantile(0.75)
    q1 = train.quantile(0.25)
    iqr = q3-q1

    for col in df.columns:
        index = df[(df[col] > q3[col]+x*iqr[col]) | (df[col] < q1[col]-x*iqr[col])].index
        if(len(index)):
            print(col , "에서 이상치 검출,,," , len(index) , "개 발견")
            print("---이상치 목록---")
            for i in index:
                print(df.loc[i][col])
            print("---끝---\n")
            sum +=len(index)
        df = df.drop(index)
        tmp_x = tmp_x.drop(index)
    #총 이상치 개수
    print(sum)

    


delete_outlier(500)

introelapse 에서 이상치 검출,,, 96 개 발견
---이상치 목록---
133690.0
27490.0
36976.0
285418.0
258417.0
18490.0
24796.0
92380.0
27490.0
79547.0
19342.0
54389.0
285418.0
37754.0
26949.0
114761.0
36779.0
75350.0
60489.0
233095.0
66300.0
34032.0
53336.0
93063.0
26635.0
123210.0
92501.0
22752.0
19723.0
258417.0
32899.0
148457.0
148457.0
71311.0
16897.0
71311.0
20267.0
75130.0
86334.0
75130.0
47502.0
855030.0
30202.0
32620.0
53693.0
36282.0
39423.0
84045.0
45389.0
16198.0
39313.0
135006.0
62218.0
22752.0
20934.0
66300.0
69661.0
70563.0
17779.0
17667.0
68422.0
82447.0
91860.0
31852.0
53182.0
20172.0
47502.0
35289.0
74654.0
43050.0
22365.0
31852.0
30645.0
76645.0
18189.0
20267.0
190936.0
16935.0
34972.0
86334.0
16897.0
35703.0
65343.0
18897.0
123210.0
75045.0
855030.0
24796.0
30202.0
92501.0
61605.0
36976.0
92602.0
82944.0
817147.0
19723.0
---끝---

testelapse 에서 이상치 검출,,, 14 개 발견
---이상치 목록---
42166.0
127962.0
407208.0
240022.0
474572.0
31721.0
71644.0
474572.0
240022.0
39055.0
374744.0
61544.0
42166.0
92462

In [10]:
tmp_x.shape

(14866, 68)

In [11]:
train=tmp_x

In [12]:
#train을 target과 feature로 나눠줍니다.
#train_ = train.dropna(axis=0)
train_= IterativeImputer().fit_transform(train.iloc[:, :-1])
train_= pd.DataFrame(train_)
test= IterativeImputer().fit_transform(test)
test= pd.DataFrame(test)
#train_ = train

In [13]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14866 entries, 0 to 14865
Data columns (total 67 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       14866 non-null  float64
 1   1       14866 non-null  float64
 2   2       14866 non-null  float64
 3   3       14866 non-null  float64
 4   4       14866 non-null  float64
 5   5       14866 non-null  float64
 6   6       14866 non-null  float64
 7   7       14866 non-null  float64
 8   8       14866 non-null  float64
 9   9       14866 non-null  float64
 10  10      14866 non-null  float64
 11  11      14866 non-null  float64
 12  12      14866 non-null  float64
 13  13      14866 non-null  float64
 14  14      14866 non-null  float64
 15  15      14866 non-null  float64
 16  16      14866 non-null  float64
 17  17      14866 non-null  float64
 18  18      14866 non-null  float64
 19  19      14866 non-null  float64
 20  20      14866 non-null  float64
 21  21      14866 non-null  float64
 22

In [14]:
norm=Normalizer()
sc=StandardScaler()
mm_scaler = MinMaxScaler()
train_S = mm_scaler.fit_transform(train_)
train_S = pd.DataFrame(train_S)
test= mm_scaler.transform(test) 
test = pd.DataFrame(test)
pca = PCA(n_components=60)
pca.fit(train_S)
#pca.fit(test)
train_pca = pca.transform(train_S)
test_pca = pca.transform(test)

In [15]:
train_pca.shape

(14866, 60)

In [16]:
test_pca.shape

(35452, 60)

In [17]:
pca_columns=['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',
            'pca_component_31','pca_component_32','pca_component_33',
            'pca_component_34','pca_component_35','pca_component_36',
            'pca_component_37','pca_component_38','pca_component_39',
            'pca_component_40','pca_component_41',
            'pca_component_42','pca_component_43','pca_component_44',
            'pca_component_45','pca_component_46','pca_component_47',
            'pca_component_48','pca_component_49','pca_component_50',
            'pca_component_51','pca_component_52','pca_component_53',
            'pca_component_54','pca_component_55','pca_component_56',
            'pca_component_57','pca_component_58','pca_component_59',
            'pca_component_60']
trainDF_pca = pd.DataFrame(train_pca, columns=pca_columns)

In [18]:
pca_columns=['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',
            'pca_component_31','pca_component_32','pca_component_33',
            'pca_component_34','pca_component_35','pca_component_36',
            'pca_component_37','pca_component_38','pca_component_39',
            'pca_component_40','pca_component_41',
            'pca_component_42','pca_component_43','pca_component_44',
            'pca_component_45','pca_component_46','pca_component_47',
            'pca_component_48','pca_component_49','pca_component_50',
            'pca_component_51','pca_component_52','pca_component_53',
            'pca_component_54','pca_component_55','pca_component_56',
            'pca_component_57','pca_component_58','pca_component_59',
            'pca_component_60']
testDF_pca = pd.DataFrame(test_pca, columns=pca_columns)

In [19]:
trainDF_pca

,pca_component_1,pca_component_2,pca_component_3,pca_component_4,pca_component_5,pca_component_6,pca_component_7,pca_component_8,pca_component_9,pca_component_10,pca_component_11,pca_component_12,pca_component_13,pca_component_14,pca_component_15,pca_component_16,pca_component_17,pca_component_18,pca_component_19,pca_component_20,pca_component_21,pca_component_22,pca_component_23,pca_component_24,pca_component_25,pca_component_26,pca_component_27,pca_component_28,pca_component_29,pca_component_30,pca_component_31,pca_component_32,pca_component_33,pca_component_34,pca_component_35,pca_component_36,pca_component_37,pca_component_38,pca_component_39,pca_component_40,pca_component_41,pca_component_42,pca_component_43,pca_component_44,pca_component_45,pca_component_46,pca_component_47,pca_component_48,pca_component_49,pca_component_50,pca_component_51,pca_component_52,pca_component_53,pca_component_54,pca_component_55,pca_component_56,pca_component_57,pca_component_58,pca_component_59,pca_component_60
0,0.975844,-0.849040,-0.449756,-0.093927,-0.182557,-0.165392,0.070846,0.496128,0.386650,-0.126737,-1.257473,0.575315,0.434520,-0.062876,-0.623076,0.232082,0.191578,-0.404348,0.301435,0.189200,-0.147019,0.991144,0.746637,0.344999,-0.064743,-0.110826,-0.180809,-0.506401,0.400112,0.087471,0.318924,0.031629,-0.256017,0.296327,-0.186292,-0.538610,-0.073358,0.098304,0.412452,0.065145,-0.290381,0.333871,-0.271612,0.045138,0.153259,-0.199844,0.002390,0.177818,0.069702,0.130851,-0.194289,0.205548,0.613681,-0.403636,-0.181894,-0.043327,0.663560,-0.162784,0.003330,0.002229
1,-0.097540,1.070209,-0.345162,0.649932,-0.645498,-0.450579,0.482783,0.033034,0.097838,-0.337704,0.405525,-0.383343,0.529061,0.540464,0.122609,-0.531378,0.219062,0.119355,-0.258586,-0.469928,0.119269,-0.186329,0.132472,-0.120059,-0.194602,0.096450,-0.528336,-0.057556,-0.250093,-0.058205,0.138437,0.111587,0.253988,0.139218,-0.039475,0.418770,0.006831,0.119756,0.185948,-0.059458,-0.260811,0.018911,-0.170941,0.164356,0.003301,-0.027163,-0.072312,0.244248,-0.080472,0.042804,0.136778,0.040994,-0.053384,0.193984,0.001617,-0.097608,-0.142981,0.179274,-0.007590,0.017453
2,-0.062698,0.123906,0.692700,-0.343591,-0.631302,0.379851,-0.745884,-0.102008,-0.061753,0.034882,-0.468788,0.263868,-0.237340,0.664697,0.373606,0.242403,-0.380837,-0.539900,-0.049873,-0.555181,-0.386182,0.075944,-0.103696,0.153574,-0.319609,0.383972,-0.412125,-0.117901,0.124376,-0.075333,0.411438,-0.130025,-0.165293,0.040950,-0.241860,0.126844,-0.446726,0.029402,0.397747,-0.297867,-0.402728,0.269237,0.284606,-0.144443,0.085410,0.048634,-0.113289,-0.296556,-0.107683,0.444340,0.596300,-0.170960,0.126761,0.124576,-0.000386,-0.046724,-0.097569,-0.114458,-0.003667,-0.010568
3,0.707897,-0.276690,-0.669419,-0.450807,0.137904,-0.056625,-0.174878,-0.209605,-0.370779,-0.077341,-0.140569,0.255594,-0.050920,-0.021271,0.461302,-0.277523,0.048303,-0.045848,-0.133716,0.214009,-0.014421,0.032340,-0.082002,-0.007083,0.428719,-0.337194,-0.097496,0.699195,0.147866,-0.056049,-0.105380,0.357239,-0.256590,0.048545,0.509214,-0.191885,0.045715,0.026999,-0.216988,0.057559,0.077849,0.056427,-0.122487,0.228714,0.062074,0.109644,-0.131574,0.024332,0.364879,-0.012539,0.192818,-0.130503,-0.046496,-0.177410,0.122775,0.117759,-0.153080,-0.096977,-0.018907,0.012825
4,0.818656,-0.357402,0.698019,-0.278768,0.768263,0.282399,-0.041292,-0.223051,-0.214563,0.283922,-0.615209,-0.131657,0.144714,0.260441,-0.347663,0.608304,0.174779,-0.084920,0.023947,0.116035,0.037940,-0.080690,-0.005822,0.666252,0.239756,-0.205328,0.232508,-0.187008,-0.209225,0.166978,0.248545,-0.076210,-0.193932,0.248585,0.346743,-0.285652,0.035704,0.096440,-0.093758,-0.161723,-0.138741,0.027731,-0.104397,-0.323677,0.320502,-0.205224,-0.280981,-0.318296,-0.076208,0.182631,-0.041035,0.164894,0.039448,0.115309,-0.119442,0.031047,-0.088997,0.017984,-0.233542,0.760557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [20]:
testDF_pca

,pca_component_1,pca_component_2,pca_component_3,pca_component_4,pca_component_5,pca_component_6,pca_component_7,pca_component_8,pca_component_9,pca_component_10,pca_component_11,pca_component_12,pca_component_13,pca_component_14,pca_component_15,pca_component_16,pca_component_17,pca_component_18,pca_component_19,pca_component_20,pca_component_21,pca_component_22,pca_component_23,pca_component_24,pca_component_25,pca_component_26,pca_component_27,pca_component_28,pca_component_29,pca_component_30,pca_component_31,pca_component_32,pca_component_33,pca_component_34,pca_component_35,pca_component_36,pca_component_37,pca_component_38,pca_component_39,pca_component_40,pca_component_41,pca_component_42,pca_component_43,pca_component_44,pca_component_45,pca_component_46,pca_component_47,pca_component_48,pca_component_49,pca_component_50,pca_component_51,pca_component_52,pca_component_53,pca_component_54,pca_component_55,pca_component_56,pca_component_57,pca_component_58,pca_component_59,pca_component_60
0,-1.291459,-0.133682,0.375087,0.228188,0.837859,0.286678,0.390917,0.011819,0.899838,0.481268,0.018483,0.031959,0.013420,0.655870,0.586976,-0.182148,-0.107134,-0.138597,0.152940,-0.147806,0.602302,0.101780,-0.096815,0.353810,0.480179,0.084609,0.344190,0.331715,-0.163053,-0.216375,-0.148758,-0.106034,-0.022532,-0.136734,-0.040558,0.473730,0.053361,0.014594,0.445769,-0.233983,-0.124626,0.146827,-0.282971,0.024954,-0.028362,-0.022293,0.240853,-0.089319,0.088629,0.091853,0.073751,0.361966,0.003655,-0.046748,-0.021412,0.041710,-0.023055,-0.139504,0.013258,0.001084
1,-0.747445,-0.754210,-0.232250,-0.266762,-0.029377,0.286953,-0.195625,-0.034489,-0.314329,0.053264,-0.389798,-0.751942,-0.258479,-0.219941,0.105254,0.147618,-0.141655,0.170024,-0.353199,-0.129708,0.306056,-0.299988,0.188964,0.153408,-0.355619,0.196657,0.462759,0.144182,-0.209939,0.052885,-0.001352,-0.290739,-0.130490,-0.002776,-0.029368,0.112458,-0.104095,-0.023216,-0.006244,0.005470,-0.213698,0.065578,0.190204,0.319378,0.201236,0.449590,0.198724,0.064815,-0.068930,-0.091927,0.076623,0.214930,0.052651,-0.115480,0.029574,-0.044870,-0.227657,0.060231,0.811284,-0.027046
2,0.009353,-0.799898,0.801532,0.790843,0.519694,-0.406877,0.051385,-0.775615,-0.423695,0.508180,-0.299603,-0.004020,0.263506,-0.010164,-0.256170,-0.040528,-0.203241,-0.229752,-0.060048,-0.412398,-0.117699,0.162628,0.046187,0.047071,0.296655,0.246454,0.051797,-0.282199,0.082590,0.046111,-0.171099,0.075631,-0.171670,-0.027055,-0.128962,0.321213,-0.453817,0.005053,0.210630,0.008978,-0.391799,-0.039550,0.075772,0.072065,0.155182,0.308171,0.201630,-0.077335,-0.034362,0.133393,0.108994,-0.186331,-0.025160,0.027036,-0.172852,0.057038,0.223843,-0.029483,-0.021900,-0.003331
3,-0.768920,0.331677,-0.504184,-0.479350,0.234849,0.177773,-0.802874,0.783492,0.323313,-0.287743,0.803818,0.341632,0.143492,0.061048,-0.000819,-0.073759,0.087844,-0.003891,-0.052786,-0.424927,0.384545,-0.013177,-0.483651,-0.097883,-0.037212,0.448491,-0.062550,0.272037,0.159948,-0.023798,-0.185544,-0.260644,0.210972,0.067796,-0.224890,-0.177048,-0.254228,-0.018310,-0.095555,-0.008265,-0.063761,0.173322,-0.285651,0.067990,0.048359,0.135070,0.406924,-0.003656,0.096381,-0.203573,-0.011421,0.071578,-0.329439,-0.146006,-0.048601,0.105722,-0.087006,0.076522,-0.045892,0.017463
4,0.875185,-1.280896,1.083187,0.800335,0.196743,-0.262487,0.108799,0.530103,-0.073543,-0.131011,0.251585,-0.440660,0.633692,-0.304652,0.417672,0.080192,-0.003707,-0.133398,0.204882,-0.155195,-0.295379,0.002585,-0.029416,-0.232105,-0.022621,0.198779,0.371129,-0.055986,-0.108314,-0.057246,-0.116474,0.292743,0.240769,-0.321642,-0.190794,-0.089927,-0.029761,-0.328214,0.007386,0.202882,0.092077,-0.353887,0.013624,0.424542,-0.527741,0.195612,-0.091361,-0.023433,0.336731,-0.161314,0.202391,-0.018866,0.304808,-0.282368,0.455711,0.198637,0.099631,0.131293,-0.267616,-0.120001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [21]:
y_target=train['nerdiness']

In [22]:
start_time=time.time()
lgb= lgb.LGBMClassifier(
            n_estimators=1000
        )
parametes= {
    'num_leaves':[20,40,60,80,100],'min_child_samples':[5,10,15],
    'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]
}
clf=RandomizedSearchCV(lgb,parametes,scoring='roc_auc',n_iter=100)
clf.fit(X=trainDF_pca,y=y_target)
end_time=time.time()
print('소요 시간 : ')
print(end_time-start_time)

소요 시간 : 
2568.7646255493164


In [23]:
# 재학습 위한 공간

In [24]:
#lgbm_clf.fit(train_x, train_y)
lgbm_pred = clf.predict(testDF_pca)

In [25]:
submission = pd.read_csv('competition_data/sample_submission.csv')

submission

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
35447,35447,-1
35448,35448,-1
35449,35449,-1
35450,35450,-1


In [26]:
submission["nerdiness"] = lgbm_pred

In [27]:
submission

,index,nerdiness
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
35447,35447,1
35448,35448,1
35449,35449,1
35450,35450,0


In [28]:
submission.to_csv("baseline_notNA-iterative-withMM-PCA60-out1.csv", index = False)